# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [1]:
import pandas as pd 
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, sum, col, desc, avg
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [2]:
spark = SparkSession.builder.appName("tarea").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/05 20:46:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [3]:
df = spark.read.csv("datosTarea.csv", header=True, inferSchema=True)
df.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [4]:
df.filter((df['Name'].startswith('M')) & (df["Number of Employees"].between(4000, 7000))).select("Name", "Number of Employees").show()

+--------------------+-------------------+
|                Name|Number of Employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [5]:
resultados = df.filter(~df["Country"].startswith("b") & ~df["Country"].startswith("s") & ~df["Country"].startswith("m") & (df["Networth"] <= 500000)).select("Name", "Country", "Networth").show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|      Pacheco-Spears|              Sweden|   88435|
|         Harrell LLC|          Guadeloupe|  251274|
|         Jenkins Inc|        South Africa|  139725|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|          Pineda-Cox|             Bolivia|  150880|
|Baker, Mccann and...|               Kenya|  188370|
|            Hahn PLC|             Belarus|  427224|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  1

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [6]:
@pandas_udf('double')
def resta_media(series: pd.Series) -> pd.Series:
    promedio = series.mean()
    return series - promedio
df.withColumn("profit-mean", resta_media(df['profit'])).show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+-----------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|profit-mean|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+-----------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|     -194.0|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|     -318.0|
|    3|0bFED1AD

/opt/conda/envs/myenv/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [7]:
# Agrupar por industry y encontrar las empresas con el profit más alto
result_df = df.groupBy("Industry").agg({"profit": "max"}).withColumnRenamed("max(profit)", "max_profit")

# Ordenar por profit de manera descendente y mostrar los tres primeros
result_df = result_df.orderBy(desc("max_profit")).limit(3)

# Mostrar el resultado
result_df.show()

+--------------------+----------+
|            Industry|max_profit|
+--------------------+----------+
|  Legislative Office|     19363|
|Museums / Institu...|     19079|
|             Farming|     18850|
+--------------------+----------+



>Agrupa por industry y calcula el promedio de empleados que tienen

In [8]:
result_df = df.groupBy("Industry").agg(avg("Number of employees").alias("Promedio Empleados"))
result_df.show()

+--------------------+------------------+
|            Industry|Promedio Empleados|
+--------------------+------------------+
|Primary / Seconda...| 6457.666666666667|
|     Broadcast Media|            2589.0|
|           Wholesale|            5010.0|
|Investment Manage...|            3133.5|
|    Food / Beverages|            9011.0|
|  Gambling / Casinos|            4873.0|
|Logistics / Procu...|            4155.0|
|            Maritime|             769.0|
|            Wireless|            6146.0|
|Education Management|             339.0|
|       Arts / Crafts|            2800.0|
|           Insurance|            1215.0|
|  Financial Services|            5157.0|
|Business Supplies...|            9097.0|
|Consumer Electronics|            5022.0|
|       Public Safety|            5287.0|
|Information Techn...|            3934.0|
|Civic / Social Or...|            2442.0|
|      Consumer Goods|            9069.0|
|Glass / Ceramics ...|            4952.0|
+--------------------+------------

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [9]:
# Crear una vista temporal para utilizar Spark SQL
df.createOrReplaceTempView("companies")

# Ejecutar la consulta SQL
result = spark.sql("SELECT COUNT(*) as count FROM companies WHERE founded > 2000")

# Mostrar el resultado
result.show()

+-----+
|count|
+-----+
|   38|
+-----+



### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [10]:
# Ensamblar las características en un solo vector
assembler = VectorAssembler(
    inputCols=["Number of employees", "Networth", "stock_price"],
    outputCol="features"
)

# Crear el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="profit")

# Crear el pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Dividir los datos en conjunto de entrenamiento y prueba
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed=123)

# Ajustar el modelo utilizando los datos de entrenamiento
model = pipeline.fit(trainingData)

# Realizar predicciones en los datos de prueba
predictions = model.transform(testData)

# Mostrar las predicciones y los valores reales
predictions.select("features", "profit", "prediction").show()

23/12/05 20:46:39 WARN Instrumentation: [170354b5] regParam is zero, which might cause numerical instability and overfitting.
23/12/05 20:46:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/05 20:46:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+--------------------+------+------------------+
|            features|profit|        prediction|
+--------------------+------+------------------+
|[5287.0,608005.0,...| 17121|12940.376776265635|
|[7832.0,900679.0,...|  5700|11380.915745440328|
|[2862.0,329130.0,...| 17043|13036.554037024318|
|[9079.0,1044084.0...| 17809|11517.630105127631|
|[7961.0,915514.0,...|  8745|11771.946946780567|
|[5038.0,579370.0,...| 11608|12820.101474409708|
|[4155.0,477824.0,...| 12282|11677.365493735513|
|[365.0,41975.0,43.0]| 12456|13285.892797461425|
|[7443.0,855944.0,...|  9688|11682.993614733978|
|[2610.0,300150.0,...| 16658|13206.745221202744|
|[1312.0,150880.0,...| 16897|13094.795107266149|
|[3715.0,427224.0,...|  5730|11493.145422669353|
|[9067.0,1042704.0...| 18075|11448.300782484934|
|[9443.0,1085945.0...| 11514|13012.814723289286|
|[1418.0,163070.0,...| 18850|12928.995793407568|
|[9097.0,1046154.0...| 12498|11424.168244223714|
|[869.0,99934.0,15.0]|  7787|11613.601961796889|
|[8433.0,969794.0,..

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [11]:
# Convertir el DataFrame de Spark a Pandas
predictions_pandas = predictions.select("features", "profit", "prediction").toPandas()

# Guardar el DataFrame de Pandas en un archivo CSV
predictions_pandas.to_csv("predictions.csv", index=False)